In [1]:
# https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html
# !pip install gspread oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import *
import pandas as pd
import numpy as np
import time
import geopandas as gpd
import datetime as dt

In [2]:
first_date = '2020-08-01'
last_date = '2020-12-16'

weekday_mapping = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}

alldates = pd.date_range(start='2020-01-01', end='2020-12-31').tolist()
datelist = [str(i.date()) for i in alldates]
weekdays = [weekday_mapping[i.date().weekday()] for i in alldates]
weeklist = [i//7 for i in range(len(alldates))]
DATE_DF = pd.DataFrame({'date':datelist,'day':weekdays,'week':weeklist})
DATE_DF = DATE_DF.loc[(DATE_DF['date']<=last_date)&(DATE_DF['date']>=first_date)].copy()
DATERANGE = DATE_DF['date'].tolist()

date_dict = {DATERANGE[i]:i for i in range(len(DATERANGE))}

In [5]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('../client_secret.json', scope)
client = gspread.authorize(creds)

In [20]:
# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
referen_sheet = client.open("00_Reference")
# summary_sheet = client.open("00_Summary")
# summary_data_sheet = summary_sheet.worksheet('summary')
ref_data = pd.DataFrame(referen_sheet.worksheet('reference'
                                    ).get_all_records())#.iloc[:,:-1]
ref_data = ref_data[['college', 'state', 'ipeds_id', 'status', 'data_url', 'notes', 'Mode of Teaching']].copy()
all_data = ref_data.loc[~ref_data.status.isin(['not publishing data', 'cannot find data', 'incomplete'])].copy()
ihe_df = pd.read_csv('../data/campus_covid_data_fall_2020.csv')

In [25]:
to_add = all_data.loc[~all_data['ipeds_id'].isin(ihe_df['ipeds_id'])].copy()

In [28]:
to_email = to_add.loc[to_add['notes'].str.contains('to email')].copy()
to_add = to_add.loc[~to_add['notes'].str.contains('to email')].copy()

In [33]:
ihe_df.ipeds_id.nunique()+to_add.shape[0]

1400

In [6]:
# completed_df = ref_data.loc[ref_data['status'].isin(['completed'])].copy()
# all_data = ref_data.loc[ref_data['status'].isin(['completed','cumulative only'])].copy()
# to_add = all_data.copy()

# Watch out, the next step takes several hours to pull
(there's almost certainly faster ways to do this, but I add sleep steps in between each new sheet)

In [35]:
unique_states = to_add.state.unique()
start = 0
unique_states

array(['Illinois', 'Kansas', 'Louisiana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'Ohio', 'Oklahoma', 'Oregon', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Alaska', 'Arkansas', 'California',
       'Colorado', 'Georgia', 'Indiana', 'Iowa', 'Maine', 'Maryland',
       'Michigan', 'South Carolina', 'South Dakota', 'Tennessee',
       'Wisconsin', 'Alabama', 'Connecticut', 'Florida', 'Hawaii',
       'Kentucky', 'Massachusetts', 'Minnesota', 'Washington',
       'West Virginia', 'Wyoming'], dtype=object)

In [37]:
add_dfs = []
nope_ipeds = []
# for st in ['Indiana']: # as an example, just look at indiana
for sit,st in enumerate(unique_states[start:]):
    state_sheet = client.open(st)
    ipdfs = to_add.loc[to_add['state']==st]['ipeds_id'].unique()
    print(st,"-",len(ipdfs),"schools left",time.strftime("%H:%M:%S", time.localtime()))
    for ip in ipdfs:
        try:
            print("\t",to_add.loc[to_add['ipeds_id']==ip].college.values[0],
                  time.strftime("%H:%M:%S", time.localtime()))
            add_dfs.append(pd.DataFrame(state_sheet.worksheet(str(ip)).get_all_records()))
            time.sleep(15)
        except:
            print('\t\tcould not do',ip)
            nope_ipeds.append(ip)

    time.sleep(30)
    start += 1

Illinois - 29 schools left 09:55:33
	 American Academy of Art 09:55:33
	 Benedictine University 09:55:50
	 Danville Area Community College 09:56:06
	 Dominican University 09:56:22
	 Governors State University 09:56:38
	 Greenville University 09:56:55
	 Heartland Community College 09:57:10
	 John A Logan College 09:57:26
	 John Wood Community College 09:57:42
	 Judson University 09:57:57
	 Kankakee Community College 09:58:13
	 Lake Land College 09:58:29
	 Lewis University 09:58:44
	 Methodist College 09:59:00
	 Millikin University 09:59:16
	 North Central College 09:59:31
	 North Park University 09:59:47
	 Olivet Nazarene University 10:00:02
	 Prairie State College 10:00:18
	 Quincy University 10:00:33
	 Rasmussen College-Illinois 10:00:49
	 Resurrection University 10:01:04
	 Roosevelt University 10:01:20
	 Rosalind Franklin University of Medicine and Science 10:01:35
	 Saint Xavier University 10:01:51
	 Shawnee Community College 10:02:06
	 University of Saint Mary of the Lake 10:02:22


In [39]:
curr_df = pd.concat(add_dfs).reset_index().iloc[:,1:]
# curr_df.to_csv('../data/supplemental_colleges_June7.tsv',sep='\t',index=False)

In [43]:
curr_df.loc[curr_df['positive_tests']==' ','positive_tests'] = np.nan
curr_df['ipeds_id'] = curr_df['ipeds_id'].astype(str)
curr_df = curr_df.loc[curr_df['date']<=last_date]

curr_df['total_tests'] = curr_df['total_tests'].astype(str)
curr_df['positive_tests'] = curr_df['positive_tests'].astype(str)

curr_df['total_tests'] = curr_df['total_tests'].str.replace(',','')
curr_df['positive_tests'] = curr_df['positive_tests'].str.replace(',','')

In [45]:
curr_df.loc[curr_df['total_tests']=='','total_tests'] = np.nan
curr_df.loc[curr_df['positive_tests']=='','positive_tests'] = np.nan

In [51]:
curr_df.loc[curr_df['total_tests']=='4200+','total_tests'] = 4200.0
curr_df.loc[curr_df['positive_tests']=='?','positive_tests'] = 1.0

curr_df['total_tests'] = curr_df['total_tests'].astype(float)
curr_df['positive_tests'] = curr_df['positive_tests'].astype(float)
curr_df['total_tests'] = np.nan_to_num(curr_df['total_tests'])
curr_df['positive_tests'] = np.nan_to_num(curr_df['positive_tests'])

In [52]:
curr_df = curr_df[['date', 'total_tests', 'positive_tests', 'college', 'url', 'ipeds_id', 'notes']].copy()
curr_df = curr_df.merge(DATE_DF,how='left',on='date')

In [53]:
curr_df.loc[curr_df['notes']=='','notes'] = np.nan
curr_df = curr_df.loc[curr_df['date']>'2020-07-30'].copy()

In [54]:
print(curr_df['college'].nunique())
print(curr_df.shape)
curr_df.head()

571
(78936, 9)


,date,total_tests,positive_tests,college,url,ipeds_id,notes,day,week
0,2020-08-01,0.0,0.0,American Academy of Art,https://www.aaart.edu/covid-19/,142887,NaN,Saturday,30
1,2020-08-02,0.0,0.0,American Academy of Art,https://www.aaart.edu/covid-19/,142887,NaN,Sunday,30
2,2020-08-03,0.0,0.0,American Academy of Art,https://www.aaart.edu/covid-19/,142887,NaN,Monday,30
3,2020-08-04,0.0,0.0,American Academy of Art,https://www.aaart.edu/covid-19/,142887,NaN,Tuesday,30
4,2020-08-05,0.0,0.0,American Academy of Art,https://www.aaart.edu/covid-19/,142887,NaN,Wednesday,31


In [55]:
# curr_df.to_csv('../data/ihe_df_googlesheets2.csv',index=False)